In [1]:
import pandas as pd
import mysql.connector
from mysql.connector import Error
import gerarJsonCsv
import subprocess

In [2]:
planta = "shopping-vitoria"
gateway_id = "GW000059"

In [3]:
# gerar o arquivo de parametros
gerarJsonCsv.generate_json_csv(planta)

/home/lucas/RepositorioProjetosCompletos/ansimble_update_gateways/gerarJsonCsv.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  modulos_data = pd.read_sql(f"\
/home/lucas/RepositorioProjetosCompletos/ansimble_update_gateways/gerarJsonCsv.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ativos_data = pd.read_sql(f"\


                  macaddress versao        aws_topic datetime_ultimo_envio  \
idModulo                                                                     
U2N000149  80:65:99:c2:22:d4   3.00  U2N000149/dados   2025-09-25 15:55:19   
U2N000150  80:65:99:c2:22:f4   3.00  U2N000150/dados   2025-09-25 15:52:00   
U2N000151  80:65:99:c2:23:04   3.00  U2N000151/dados   2025-09-04 22:31:30   
U2N000152  80:65:99:c2:22:c0   3.00  U2N000152/dados   2025-09-04 22:30:39   

           TC_ratio Rog_ratio  corrente_l1  corrente_l2  corrente_l3  \
idModulo                                                               
U2N000149     10000      None            0            1            2   
U2N000150     10000      None            0            1            2   
U2N000151     10000      None            0            1            2   
U2N000152     10000      None            0            1            2   

           tensao_l1  ...  sentido_tc2  sentido_tc3  instalado  Ativo  \
idModulo              ...

In [4]:
mydb = mysql.connector.connect(
    host="ultronline.cibxpi1gohmx.us-east-1.rds.amazonaws.com",
    user="root",
    password="2neuron2",
    database='projeto_ultron_teste'
)
mycursor = mydb.cursor()

In [5]:
gateways_data = pd.read_sql(f"\
SELECT * FROM Gateways where Plantas_idPlanta='{planta}'", mydb)

/tmp/ipykernel_87681/412794979.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  gateways_data = pd.read_sql(f"\


In [6]:
gateways_data

,idGateway,wifi_ssid,wifi_passkey,mqtt_ip,mqtt_user,mqtt_passkey,mqtt_port,cript_key,rasp_user,rasp_passkey,chip_numero,aws_thing,ipv6,macaddress,key_credenciais,Plantas_idPlanta,Real_vnc_login
0,GW000059,GW000059,g94onof5,10.42.0.1,GW000059,fZZDCkur,1883,RAInPFAdEmwHzaWl,gateway,iViNSXYC,+5527998472751,GW000059,2804:18:5081:b4f:29b3:d995:9c6e:f3d5,2c:cf:67:3c:1b:30,None,shopping-vitoria,nao@tem
1,GW000060,GW000060,fQiWrl2R,10.42.0.1,GW000060,k7NU7tEp,1883,KE7ewM8T4eWemkYy,gateway,T9yzPmGh,+5527995893913,GW000060,2804:18:5080:f050:3d7c:5a33:9eb7:2cd,d8:3a:dd:c6:90:9e,None,shopping-vitoria,nao@tem


In [7]:
row = gateways_data[gateways_data['idGateway'] == gateway_id]
ipv6 = row["ipv6"].values[0]
rasp_passkey = row["rasp_passkey"].values[0]

In [8]:
print(ipv6)

2804:18:5081:b4f:29b3:d995:9c6e:f3d5


In [9]:
result = subprocess.run(
            [
                "ssh-keyscan", "-p", "30022",
                "-H", ipv6, ">>", "~/.ssh/known_hosts"], capture_output=True, text=True, check=True
        )

In [10]:
result

CompletedProcess(args=['ssh-keyscan', '-p', '30022', '-H', '2804:18:5081:b4f:29b3:d995:9c6e:f3d5', '>>', '~/.ssh/known_hosts'], returncode=0, stdout='|1|K6rPUxubaedF21jKb739ORjo7b4=|y8i5+xVZ9QbzJ8qZPHh0D5jrUrc= ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQC2JfmYoQOvo5OSw/L6F3dCr+87wWo4zNGaVWCHXNswgFi50uJt3oWRVPi6LiOBFpC5jbsJjd8JboTnSuKlwc/Aggg4eZNpK+U+xgiEktoL4rqAo1o4Wxq7Jt6mTiXdApvU2pMTMPkpOxI6nGgE+hqTyIDsMc9t3ieTUR1qMEoayO9GA8Ok4YHt2Uq/lbUf7dfPP+Gjq4SVujp9nNjNZDsVfTERKYORODixb7rUiQzUYcxgMXo8f8bCmll9u41IXZ2ZBKMjIEgFKy9MAGTMS9NqwsvMM8kaHodlwTcD4JfhexcXHnTzj5aX/0WbFaXbk5RXqUGIpZqevUJTv0mmmn3K376Lzjvoh5dlyALZ4KcSU/7TLmtNpMGNNFHDxmbpghKCdIjt6ftXb1Ke9cCjRu6yrJINPVlOMEXz6btdwsrwW+GyGAZT481NzXO2G+gAXNyCbD1ETxlJgTyxArKEQOtm9hiL1vkHRVy+Wul08/GIkLNLagN+SMRho7EcFBLlp1M=\n|1|xh8IFjwKvPTG0kXT+pc21Fe8azU=|acDi099pOvn/3i/5fzq/ovchxvM= ecdsa-sha2-nistp256 AAAAE2VjZHNhLXNoYTItbmlzdHAyNTYAAAAIbmlzdHAyNTYAAABBBDqgWjHL9m/frtGGWqum2UH4qVZIhhzDNxZSPJOXCByEpNtPVJ0RMH6pL3+lukpYjPT0e/iqjwDBQf3fOWZHYfc=\n|1|/0q7lSTzK

In [11]:
file_text = f"[myhosts]\n\
servidor1 ansible_host={ipv6} ansible_port=30022 ansible_user=gateway ansible_ssh_pass={rasp_passkey} \
ansible_become_pass={rasp_passkey} ansible_ssh_common_args='-6'"

In [12]:
with open(f"inventory-{planta}.ini", "w") as f:
    f.write(file_text)

In [13]:
# escrever o playbook
playbook_text = f"""\
---
- name: move files
  hosts: myhosts
  become: true  # se precisar de sudo
  tasks:
  - name: Move ultronline.py
    ansible.builtin.copy:
      src: /home/lucas/Servidores/server_v4/Server/ultronline.py
      dest: /home/gateway/Servidores/server_v4/Server/ultronline.py
      owner: root
      group: root
      mode: '0644'
  
  - name: Move servidor_main.py
    ansible.builtin.copy:
      src: /home/lucas/Servidores/server_v4/Server/servidor_main.py
      dest: /home/gateway/Servidores/server_v4/Server/servidor_main.py
      owner: root
      group: root
      mode: '0644'
  
  - name: Move energy_analysis.py
    ansible.builtin.copy:
      src: /home/lucas/Servidores/server_v4/Server/energy_analysis.py
      dest: /home/gateway/Servidores/server_v4/Server/energy_analysis.py
      owner: root
      group: root
      mode: '0644'
  
  - name: Move parametros_ultronline.json
    ansible.builtin.copy:
      src: /home/lucas/RepositorioProjetosCompletos/ansimble_update_gateways/\
parametros_ultronline_{planta}.json
      dest: /home/gateway/Servidores/server_v4/Server/parametros_ultronline.json
      owner: root
      group: root
      mode: '0644'
  
  - name: Reiniciar ultronlink_server
    ansible.builtin.service:
      name: ultronlink_server
      state: restarted
"""

In [14]:
with open(f"playbook-{planta}.yml", "w") as f:
    f.write(playbook_text)

In [15]:
mycursor.close()
mydb.close()